IMPORT YOUR LIBRARIES

In [ ]:
from bs4 import BeautifulSoup as bf
import requests
import pandas as pd
import numpy as np
import datetime as dt

REQUESTING DATA FROM THE WEB ADDRESS AND CHECKING THE STATUS CODE

In [ ]:
#Our web address is assigned to a variable called url
url = 'https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100'
#we implementt the requests.get() method to fetch all details about our web page and assign to the resppnse variable
response = requests.get(url)
#All responses from a get() method have status codes to indicate what was contained in the feedback
#Examples: Code 200:OK, 404:Page Not Found 503:Server error etc.
print(response.status_code)

200


INSPECTING/FORMATTING OUR RESPONSE CONTENT AND ADDIGN TO A VARIABLE USING BEAUTIFUL SOUP

In [ ]:
#We fetch the content of our response and inspect/format it with the html parser
#assign to the variable soup
soup =bf(response.content,'html.parser')
#soup

In [ ]:
#Remember the 2 main tags within our html document is th head and body
#Our focus for this exercise however is the body so we assign that to a varaible
body=soup.body
#Uncomment the code below to view html for the body
#print(body.prettify())

EXTRACTING DATA TO CREATE OUR DATAFRAME

###REVIEW TITLE###

In [ ]:
#<h2 class="text_header">
#"stick to their cabin bag size limit"
#</h2>
#From our inspection we realized "stick to their cabin bag size limit" together with all OTHER titles are found in the h2 tag with class vale of text_header"
#We extract all such values with the body.find_all function
review_headers = body.find_all('h2', class_='text_header')
len(review_headers)

100

In [ ]:
#For loop to Extract text from tags
title =[reviewstr.text for reviewstr in review_headers]
len(title)

100

In [ ]:
#We create an instance of data frame with no rows or columns
df=pd.DataFrame()
#we further create a Title column and assign our values in the new list
df['Title']=title

#We realized some all records began and ended wwith "" symbols so we slice them off
#df['Title']=df['Title'].str[1:-1]
df['Title'] = df['Title'].str.replace('"', '')
df


,Title
0,baggage customer service is a joke
1,Worst business class ever
2,replicating the likes of Ryanair
3,declined to offer compensation
4,onboard like discount carriers
...,...
95,do not upgrade members based on status
96,Flight was smooth and quick
97,Absolutely hopeless airline
98,Customer Service is non existent


###REVIEWS###

In [ ]:
"""<div class="text_content" itemprop="reviewBody">
            <strong>
             <a href="https://www.airlinequality.com/verified-reviews/">
              <em>
               Not Verified
              </em>
             </a>
            </strong>
            |  The flight was comfortable enough but with an hour delay on the return leg. However, on both leg I was told I had to put my very small and expensive cabin case into the hold as the flight was full. Having done so I was not amused to see other passengers bringing much larger cases into the cabin. BA should stick to their cabin bag size limit and not inconvenience those who comply.
</div>
"""
#From our inspection we realized the reviews can be found  found in the div tag with class value of reviewBody"
#We extract all such vales with the body.find_all function
review_details = body.find_all('div', class_='text_content')
#review_details

In [ ]:
#Extract text from tags
detailer =[detailstr.text.strip() for detailstr in review_details]
#detailer

In [ ]:
#air_td=soup.find_all('td',class_='review-value')

#view = [tablestr.text for tablestr in air_table]
#len(view)

In [ ]:
#num_columns= 6
#columns = {}
#for index, value in enumerate(view):
#  column_index = index % num_columns
#  if column_index not in columns:
#    columns[column_index] =[]
#  columns[column_index].append(value)

#for col_index, col_values in columns.items():
#  print(f"Column {col_index} : {col_values}")

###CABIN TYPE###

In [ ]:
"""
  Find this tag
  <td class="review-rating-header cabin_flown">
  Seat Type
  </td>
  Then you can find its next sibling
  <td class="review-value">
  Economy Class
  </td>
"""
#Find all elements with a specific tag  td and class=cabin_flown
#Since all other values  share a similar tag and css class, we need to find use the next sibling tag to find its associated tag
elements = soup.body.find_all('td',class_='review-rating-header cabin_flown')
#list variable to hold the values
seat_type=[]
# Iterate over each element
for element in elements:
    # Perform whatever operations you need with the individual element
    # For example, find the next sibling of each element
    next_sibling = element.find_next_sibling()
    if next_sibling:
        seat_type.append(next_sibling.text)
    else:
        print("No next sibling found")

In [ ]:
# we check length of the list
len(seat_type)

100

###RATING VALUE###

In [ ]:
rating_vals = body.find_all('span',attrs={'itemprop':'ratingValue'})
val_rating=[vals.text.strip() for vals in rating_vals[1:]]

val_rating

['2',
 '1',
 '4',
 '2',
 '6',
 '1',
 '1',
 '1',
 '1',
 '1',
 '6',
 '1',
 '1',
 '10',
 '7',
 '3',
 '4',
 '8',
 '1',
 '8',
 '5',
 '10',
 '9',
 '3',
 '8',
 '8',
 '3',
 '3',
 '7',
 '3',
 '1',
 '7',
 '1',
 '3',
 '1',
 '1',
 '3',
 '10',
 '1',
 '3',
 '2',
 '1',
 '10',
 '3',
 '3',
 '2',
 '5',
 '1',
 '8',
 '1',
 '6',
 '9',
 '8',
 '3',
 '2',
 '1',
 '6',
 '3',
 '2',
 '1',
 '1',
 '1',
 '4',
 '9',
 '1',
 '9',
 '6',
 '1',
 '8',
 '6',
 '2',
 '5',
 '3',
 '6',
 '10',
 '1',
 '1',
 '6',
 '6',
 '4',
 '1',
 '8',
 '3',
 '4',
 '9',
 '9',
 '10',
 '1',
 '1',
 '7',
 '1',
 '5',
 '1',
 '9',
 '1',
 '3',
 '8',
 '1',
 '1',
 '1']

###AIRCRAFT###

In [ ]:
#Find all elements with a specific tag  td and class=review-rating-header route
#Since all other values  share a similar tag and css class, we need to find use the next sibling tag to find its associated tag
# Find all elements with a specific tag-route
aircraft=[]
for tag in soup.body.findAll('table',attrs={'class':'review-ratings'}):
  aircrafter = tag.find('td', attrs={'class':'review-rating-header aircraft'})
  if aircrafter:
    next_sibling_aircraft = aircrafter.find_next_sibling()
    aircraft.append(next_sibling_aircraft.get_text().strip())
  else:
    aircraft.append('N/A')



In [ ]:
aircraft = aircraft[1:]
aircraft

['N/A',
 'A380',
 'A320',
 'N/A',
 'A319 / A320',
 'N/A',
 'A380',
 'A320',
 'A350',
 'N/A',
 'A319 / E190',
 'N/A',
 'N/A',
 'A340',
 'Boeing 787',
 'Boeing 777-200',
 'Boeing 777-200',
 'A350',
 'N/A',
 'Boeing 777',
 'A320',
 'Boeing 777',
 'A320',
 'N/A',
 'A320',
 'Boeing 787-8 /777-200 / A320-200',
 'A321',
 'Boeing 777-300',
 'A320',
 'Boeing 777',
 'N/A',
 'N/A',
 'Boeing 777-300ER',
 'Boeing 777-300',
 'N/A',
 'N/A',
 'Boeing 787',
 'A319',
 'N/A',
 'A320',
 'A320',
 'N/A',
 'A320',
 'A350',
 'A320',
 'Boeing 777',
 'A380',
 'N/A',
 'N/A',
 'N/A',
 'A380-800',
 'N/A',
 'Boeing 777-200',
 'N/A',
 'N/A',
 'N/A',
 'A350',
 'A320',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'A350-1000',
 'A320',
 'Boeing 777 -200',
 'N/A',
 'N/A',
 'A320',
 'A320',
 'Boeing 777',
 'Boeing 777',
 'Boeing 777',
 'A350',
 'A329',
 'N/A',
 'N/A',
 'A321',
 'A320/Boeing 777',
 'Boeing 787-8',
 'N/A',
 'A350',
 'N/A',
 'A380',
 'Boeing 777',
 'A320',
 'A319',
 'N/A',
 'N/A',
 'A380',
 'N/A',
 'Boeing 787-

###ROUTE###

In [ ]:
"""
  <td class="review-rating-header route">
    Route
  </td>
  <td class="review-value">
    London Heathrow to Toulouse
  </td>
"""
"""
#Find all elements with a specific tag  td and class=review-rating-header route
#Since all other values  share a similar tag and css class, we need to find use the next sibling tag to find its associated tag
# Find all elements with a specific tag-route
elements = soup.body.find_all('td',class_='review-rating-header route')
route=[]
# Iterate over each element
for element in elements:
    # Perform whatever operations you need with the individual element
    # For example, find the next sibling of each element
    next_sibling = element.find_next_sibling()
    if next_sibling:
        route.append(next_sibling.text)
    else:
        print("No next sibling found")
"""

'\n#Find all elements with a specific tag  td and class=review-rating-header route\n#Since all other values  share a similar tag and css class, we need to find use the next sibling tag to find its associated tag\n# Find all elements with a specific tag-route\nelements = soup.body.find_all(\'td\',class_=\'review-rating-header route\')\nroute=[]\n# Iterate over each element\nfor element in elements:\n    # Perform whatever operations you need with the individual element\n    # For example, find the next sibling of each element\n    next_sibling = element.find_next_sibling()\n    if next_sibling:\n        route.append(next_sibling.text)\n    else:\n        print("No next sibling found")\n'

In [ ]:
#First, find the table tag with class review-ratings
#Find all elements with a specific tag  td and class=review-rating-header route
#Since all other values  share a similar tag and css class, we need to find use the next sibling tag to find its associated tag
# Find all elements with a specific tag-route
router=[]
for tag in soup.body.findAll('table',attrs={'class':'review-ratings'}):
  route_var = tag.find('td', attrs={'class':'review-rating-header route'})
  if route_var:
    next_sibling_route = route_var.find_next_sibling()
    router.append(next_sibling_route.get_text().strip())
  else:
    router.append('N/A')

In [ ]:
len(router)

101

In [ ]:
router = router[1:]
router

['Malaga to Boston via London',
 'Milan Linate to Miami via London Heathrow',
 'Catania to Gatwick',
 'London to Hannover',
 'Manchester to Copenhagen via London',
 'London to Valencia',
 'Singapore to London',
 'Madrid to London Heathrow',
 'Austin to London Heathrow',
 'Zurich to Phoenix via Heathrow',
 'Rotterdam to London City',
 'Kansas City to Dublin via Heathrow',
 'Tenerife to London',
 'Lisbon to San Diego',
 'London Heathrow to Mumbai',
 'Doha to London Heathrow',
 'Doha to London Heathrow',
 'London Heathrow to São Paulo',
 'London Heathrow to Nice',
 'London to Tokyo',
 'Lisbon to London',
 'London to Delhi',
 'London to Milan',
 'Madrid to London Heathrow',
 'London Heathrow to Toulouse',
 'Montreal to Venice via London Heathrow',
 'Paphos to London',
 'London to Houston',
 'London Heathrow to Brussels',
 'Barbados to London heathrow',
 'Toronto to Mumbai via London',
 'Copenhagen to Port of Spain via London',
 'Los Angeles to London',
 'Singapore to Sydney',
 'Cyprus to L

###RECOMMENDED###

In [ ]:
"""
<td class="review-rating-header recommended">
  Recommended
</td>
<td class="review-value rating-yes">
  yes
</td>
"""
# Find all elements with a specific tag-review-rating-header recommended
elements = soup.body.find_all('td',class_='review-rating-header recommended')
recommend=[]
# Iterate over each element
for element in elements:
    # Perform whatever operations you need with the individual element
    # For example, find the next sibling of each element
    next_sibling = element.find_next_sibling()
    if next_sibling:
        recommend.append(next_sibling.text)
    else:
        print("No next sibling found")

In [ ]:
len(recommend)

100

###DATE PUBLISHED###

In [ ]:
#fetch date of publishong the review
review_date = body.find_all('time', itemprop="datePublished")
date_Pub = [datestr.text for datestr in review_date]
len(date_Pub)

100

###SEAT COMFORT- STAR RATINGS###

In [ ]:
#Fetch Star Ratings
#Find all elements with a specific tag  td and class=review-rating-stars stars
#Since all other values  share a similar tag and css class, we need to find use the next sibling tag to find its associated tag
# Find all elements with a specific tag-route
"""
<td class="review-rating-stars stars">
  <span class="star fill">1</span>
  <span class="star fill">2</span>
  <span class="star fill">3</span>
  <span class="star">4</span>
  <span class="star">5</span>
</td>
"""

seart_highest_rating=[]
for tag in soup.body.findAll('table',class_='review-ratings'):
  tagger = tag.find('td',class_='review-rating-header seat_comfort')
  if tagger:
    rate_tag = tagger.find_next_sibling()
    filled_stars = rate_tag.find_all('span', class_='star fill')
    if filled_stars:
      rating = len(filled_stars)

      seart_highest_rating.append(rating)

  else:
    seart_highest_rating.append('N/A')

len(seart_highest_rating)


101

###CABIN STAFF SERVICE###

In [ ]:
cabin_highest_rating=[]
for tag in soup.body.findAll('table',class_='review-ratings'):
  tagger = tag.find('td',class_='review-rating-header cabin_staff_service')
  if tagger:
    rate_tag = tagger.find_next_sibling()
    filled_stars = rate_tag.find_all('span', class_='star fill')
    if filled_stars:
      rating = len(filled_stars)

      cabin_highest_rating.append(rating)

  else:
    cabin_highest_rating.append('N/A')

len(cabin_highest_rating)

101

###FOOD AND BEVERAGES###

In [ ]:
food_highest_rating=[]
for tag in soup.body.findAll('table',class_='review-ratings'):
  tagger = tag.find('td',class_='review-rating-header food_and_beverages')
  if tagger:
    rate_tag = tagger.find_next_sibling()
    filled_stars = rate_tag.find_all('span', class_='star fill')
    if filled_stars:
      rating = len(filled_stars)

      food_highest_rating.append(rating)

  else:
    food_highest_rating.append('N/A')

len(food_highest_rating)

101

###IN-FLIGHT ENTERTAINMENT###

In [ ]:

entertainment_highest_rating=[]
for tag in soup.body.findAll('table',class_='review-ratings'):
  tagger = tag.find('td',class_='review-rating-header inflight_entertainment')
  if tagger:
    rate_tag = tagger.find_next_sibling()
    filled_stars = rate_tag.find_all('span', class_='star fill')
    if filled_stars:
      rating = len(filled_stars)

      entertainment_highest_rating.append(rating)

  else:
    entertainment_highest_rating.append('N/A')

len(entertainment_highest_rating)

101

###GROUND SERVICE###

In [ ]:
grd_svc_highest_rating=[]
for tag in soup.body.findAll('table',class_='review-ratings'):
  tagger = tag.find('td',class_='review-rating-header ground_service')
  if tagger:
    rate_tag = tagger.find_next_sibling()
    filled_stars = rate_tag.find_all('span', class_='star fill')
    if filled_stars:
      rating = len(filled_stars)

      grd_svc_highest_rating.append(rating)

  else:
    grd_svc_highest_rating.append('N/A')

len(grd_svc_highest_rating)

101

###WIFI AND CONNECTIVITY###

In [ ]:

wifi_highest_rating=[]
for tag in soup.body.findAll('table',class_='review-ratings'):
  tagger = tag.find('td',class_='review-rating-header wifi_and_connectivity')
  if tagger:
    rate_tag = tagger.find_next_sibling()
    filled_stars = rate_tag.find_all('span', class_='star fill')
    if filled_stars:
      rating = len(filled_stars)

      wifi_highest_rating.append(rating)

  else:
    wifi_highest_rating.append('N/A')

len(wifi_highest_rating)

101

###VALUE FOR MONEY###

In [ ]:
value_highest_rating=[]
for tag in soup.body.findAll('table',class_='review-ratings'):
  tagger = tag.find('td',class_='review-rating-header value_for_money')
  if tagger:
    rate_tag = tagger.find_next_sibling()
    filled_stars = rate_tag.find_all('span', class_='star fill')
    if filled_stars:
      rating = len(filled_stars)

      value_highest_rating.append(rating)

  else:
    value_highest_rating.append('N/A')

len(value_highest_rating)

101

#POPULATE THE DATA FRAME WITH THE LISTS###

In [ ]:
df['Aircraft']=aircraft
df['Review']=detailer
df['Cabin']=seat_type
df['Route']=router
df['Seat_Comfort']=seart_highest_rating[1:]
df['Cabin_Staff_Service']=cabin_highest_rating[1:]
df['Inflight_Entertainment']=entertainment_highest_rating[1:]
df['Ground_Service']=grd_svc_highest_rating[1:]
df['Wifi_COnnectivity']=wifi_highest_rating[1:]
df['Value_for_Money']=value_highest_rating[1:]
df['Rating_Value']=val_rating
df['Recommended']=recommend
df['Pub_Date']=date_Pub
df

,Title,Aircraft,Review,Cabin,Route,Seat_Comfort,Cabin_Staff_Service,Inflight_Entertainment,Ground_Service,Wifi_COnnectivity,Value_for_Money,Rating_Value,Recommended,Pub_Date
0,baggage customer service is a joke,N/A,Not Verified | I flew from Malaga via LHR to...,Economy Class,Malaga to Boston via London,2,2,2,1,1,2,2,no,8th May 2024
1,Worst business class ever,A380,✅ Trip Verified | Milan to Miami return via L...,Business Class,Milan Linate to Miami via London Heathrow,1,2,2,2,N/A,1,1,no,8th May 2024
2,replicating the likes of Ryanair,A320,✅ Trip Verified | BA created a new low-cost s...,Economy Class,Catania to Gatwick,2,1,N/A,3,1,1,4,no,7th May 2024
3,declined to offer compensation,N/A,✅ Trip Verified | I flew with British Airway...,Economy Class,London to Hannover,3,3,N/A,1,1,1,2,no,5th May 2024
4,onboard like discount carriers,A319 / A320,✅ Trip Verified | Manchester to Copenhagen vi...,Economy Class,Manchester to Copenhagen via London,3,3,N/A,4,N/A,4,6,yes,1st May 2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,do not upgrade members based on status,A320,✅ Trip Verified | I recently had a delay on B...,Economy Class,Brussels to London,2,3,2,1,2,2,3,no,19th November 2023
96,Flight was smooth and quick,A320,"Not Verified | Boarded on time, but it took a...",Business Class,London Heathrow to Dublin,3,3,N/A,4,N/A,3,8,yes,16th November 2023
97,Absolutely hopeless airline,N/A,"✅ Trip Verified | 5 days before the flight, w...",Economy Class,London to Dublin,3,3,N/A,1,N/A,1,1,no,16th November 2023
98,Customer Service is non existent,N/A,Not Verified | \r\nWe traveled to Lisbon for ...,Economy Class,London to Lisbon,1,1,1,1,1,1,1,no,14th November 2023


In [ ]:
#Splitting the Review Cell based on the index of the Pipe character
df[['Verified','Review']] = df['Review'].str.split('|',expand=True)
df

,Title,Aircraft,Review,Cabin,Route,Seat_Comfort,Cabin_Staff_Service,Inflight_Entertainment,Ground_Service,Wifi_COnnectivity,Value_for_Money,Rating_Value,Recommended,Pub_Date,Verified
0,baggage customer service is a joke,N/A,I flew from Malaga via LHR to Boston on the...,Economy Class,Malaga to Boston via London,2,2,2,1,1,2,2,no,8th May 2024,Not Verified
1,Worst business class ever,A380,Milan to Miami return via London. Worst busi...,Business Class,Milan Linate to Miami via London Heathrow,1,2,2,2,N/A,1,1,no,8th May 2024,✅ Trip Verified
2,replicating the likes of Ryanair,A320,BA created a new low-cost subsidiary in BA E...,Economy Class,Catania to Gatwick,2,1,N/A,3,1,1,4,no,7th May 2024,✅ Trip Verified
3,declined to offer compensation,N/A,I flew with British Airways from Heathrow t...,Economy Class,London to Hannover,3,3,N/A,1,1,1,2,no,5th May 2024,✅ Trip Verified
4,onboard like discount carriers,A319 / A320,Manchester to Copenhagen via London on 30th ...,Economy Class,Manchester to Copenhagen via London,3,3,N/A,4,N/A,4,6,yes,1st May 2024,✅ Trip Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,do not upgrade members based on status,A320,I recently had a delay on British Airways fr...,Economy Class,Brussels to London,2,3,2,1,2,2,3,no,19th November 2023,✅ Trip Verified
96,Flight was smooth and quick,A320,"Boarded on time, but it took ages to get to ...",Business Class,London Heathrow to Dublin,3,3,N/A,4,N/A,3,8,yes,16th November 2023,Not Verified
97,Absolutely hopeless airline,N/A,"5 days before the flight, we were advised by...",Economy Class,London to Dublin,3,3,N/A,1,N/A,1,1,no,16th November 2023,✅ Trip Verified
98,Customer Service is non existent,N/A,\r\nWe traveled to Lisbon for our dream vaca...,Economy Class,London to Lisbon,1,1,1,1,1,1,1,no,14th November 2023,Not Verified


In [ ]:
df['Review'] = df['Review'].str.strip()
df['Verified'] = df['Verified'].str.strip()
df

,Title,Aircraft,Review,Cabin,Route,Seat_Comfort,Cabin_Staff_Service,Inflight_Entertainment,Ground_Service,Wifi_COnnectivity,Value_for_Money,Rating_Value,Recommended,Pub_Date,Verified
0,baggage customer service is a joke,N/A,I flew from Malaga via LHR to Boston on the 4t...,Economy Class,Malaga to Boston via London,2,2,2,1,1,2,2,no,8th May 2024,Not Verified
1,Worst business class ever,A380,Milan to Miami return via London. Worst busine...,Business Class,Milan Linate to Miami via London Heathrow,1,2,2,2,N/A,1,1,no,8th May 2024,✅ Trip Verified
2,replicating the likes of Ryanair,A320,BA created a new low-cost subsidiary in BA Eur...,Economy Class,Catania to Gatwick,2,1,N/A,3,1,1,4,no,7th May 2024,✅ Trip Verified
3,declined to offer compensation,N/A,I flew with British Airways from Heathrow to H...,Economy Class,London to Hannover,3,3,N/A,1,1,1,2,no,5th May 2024,✅ Trip Verified
4,onboard like discount carriers,A319 / A320,Manchester to Copenhagen via London on 30th Ap...,Economy Class,Manchester to Copenhagen via London,3,3,N/A,4,N/A,4,6,yes,1st May 2024,✅ Trip Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,do not upgrade members based on status,A320,I recently had a delay on British Airways from...,Economy Class,Brussels to London,2,3,2,1,2,2,3,no,19th November 2023,✅ Trip Verified
96,Flight was smooth and quick,A320,"Boarded on time, but it took ages to get to th...",Business Class,London Heathrow to Dublin,3,3,N/A,4,N/A,3,8,yes,16th November 2023,Not Verified
97,Absolutely hopeless airline,N/A,"5 days before the flight, we were advised by B...",Economy Class,London to Dublin,3,3,N/A,1,N/A,1,1,no,16th November 2023,✅ Trip Verified
98,Customer Service is non existent,N/A,"We traveled to Lisbon for our dream vacation, ...",Economy Class,London to Lisbon,1,1,1,1,1,1,1,no,14th November 2023,Not Verified


###EXPOERING YOUR DATASET TO CSV###

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Beautiful_Soup/sample_dataframe3.csv', 'w') as f:
    df.to_csv(f, index=False)
